In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime

from download_tools.plugins.mouselab_mdp import preprocess_mouselab_data, add_click_count_columns
from download_tools.plugins.survey_html_form import process_html_demographics
from download_tools.plugins.survey_multi_choice import score_mouselab_questionnaires, get_mouselab_quiz_name, get_quiz_passer_ids, score_row, score_generic_questionnaires
from download_tools.plugins.survey_text import preprocess_survey_text, get_old_demographics
from download_tools.plugins.utils import get_demo_string

from datetime import datetime
import json
import dill as pickle

In [2]:
# Parameters
database_key = "NEW"
participants_to_remove = [1640]
simulated = False
sessions = ["QuestPilot1", "QuestPilot2", "QuestPilot3"]
questionnaire_files = {"QuestPilot1": "questionnaires_json4", "QuestPilot2": "questionnaires_json3", "QuestPilot3": "questionnaires_json5"}
experiment_setting = "high_increasing"
COST = None
DEPTH = None
html_survey_names = {"QuestPilot2": {"0.0-4.0-18.0": "demographics"}, "QuestPilot3": {"0.0-4.0-18.0": "demographics"}}
num_parts = None
old_experiment = [True, False, False]
manual_age_mapping = {}
experiment_specific_mapping = {}
trials_per_block = {"test": 30}
ranges_to_extract = {"training": "range(20)", "test": "range(20,30)"}
mouselab_column_identifier = "name"
mouselab_mapping = {"0.0-2.*-1.*": "mouselab-quiz-pre", "0.0-4.0-2.0": "mouselab-quiz-post", "0.0-3.0-*.0": "mouselab-quiz-post", "0.0-19.*-2.*": "mouselab-quiz-post", "0.0-17.*-1.*": "mouselab-quiz-pre"}
max_attempts = {"mouselab-quiz-pre": 4, "mouselab-quiz-post": 1}
passing_score = {"mouselab-quiz-pre": 4, "mouselab-quiz-post": 0}
mouselab_quiz_solutions = {"mouselab-quiz-pre": {"Q0": "$-48 to $48", "Q1": "$1", "Q2": "The better I perform the higher my bonus will be.", "Q3": "No, the amount of cash at each node of the web may be different each time."}, "mouselab-quiz-post": {"Q0": "$-4 to $4", "Q1": "$-8 to $8", "Q2": "$-48 to $48", "Q3": "No, the cost is always $1.00.", "Q4": {"Very unmotivated": -2, "Slightly unmotivated": -1, "Neither motivated nor unmotivated": 0, "Slightly motivated": 1, "Very motivated": 2, "": 0}}}
crt_quiz_solutions = {"crt1": {".10": "intuitive", "10": "intuitive", ".05": "correct", "5": "correct", "": "no response"}, "crt2": {"100": "intuitive", "5": "correct", "": "no response"}, "crt3": {"24": "intuitive", "47": "correct", "": "no response"}, "crt4": {"9": "intuitive", "4": "correct", "": "no response"}, "crt5": {"30": "intuitive", "29": "correct", "": "no response"}, "crt6": {"10": "intuitive", "20": "correct", "": "no response"}, "crt7": {"is ahead of where he began": "intuitive", "has lost money": "correct", "": "no response"}}
ground_truth_file = "312_2_4_24"
node_classification = {"early": [1, 5, 9], "middle": [2, 6, 10], "late": [3, 4, 7, 8, 11, 12], "clicks": [1, 5, 9, 2, 6, 10, 3, 4, 7, 8, 11, 12]}
structure = "312_2_4_24"
analysis_run = "quest_first"
data_path = "/home/vfelso/github/planning-depth-differences/data"

In [3]:
# paths to use
inputs_path = Path(data_path).joinpath('inputs')
raw_data_path = Path(data_path).joinpath('raw')
processed_data_path = Path(data_path).joinpath(f'processed/{analysis_run}')
processed_data_path.mkdir(parents=True, exist_ok=True)

In [4]:
with open(inputs_path.joinpath(f"exp_inputs/rewards/{ground_truth_file}.json")) as json_file:
    ground_truths = json.load(json_file)

In [5]:
# load data
full_data = {}

# read in sessions
for run in sessions:
    for file_path in raw_data_path.glob(f"{run}/*.csv"):
        # don't want to save identifiable bonuses
        # file, information is already in data
        if "bonuses" not in str(file_path):
            file_name = file_path.stem
            curr_data_frame = pd.read_csv(file_path)
            curr_data_frame["run"] = run

            # remove participant who answered they were too young
            curr_data_frame = curr_data_frame[~curr_data_frame["pid"].isin(participants_to_remove)]

            if file_name not in full_data:
                full_data[file_name] = [curr_data_frame]
            else:
                full_data[file_name].append(curr_data_frame)

full_data = {k: pd.concat(v) for k,v in full_data.items()}

In [6]:
individual_variables = full_data["general_info"].merge(full_data["question_data"], on=["pid","run"])

time_format = '%Y-%m-%d %H:%M:%S.%f'
finished_df = individual_variables[individual_variables["endhit"].apply(lambda endhit: isinstance(endhit, str))].reset_index()
finished_df["time_diff"] = finished_df.apply(lambda row: ((datetime.strptime(row["endhit"], time_format) - datetime.strptime(row["beginhit"], time_format)).seconds % 3600 )/ 60.0, axis=1)
individual_variables = individual_variables.merge(finished_df[["time_diff", "pid", "run"]], how="left", on=["pid", "run"])

# check saved cost makes sense
if (COST is not None) and (DEPTH is not None):
    if isinstance(COST, dict):
        assert(np.all(individual_variables.apply(lambda row: row["COST"] == COST[row["codeversion"]][int(row["cond"])],axis=1)))
    else:
        unique_costs = np.unique(individual_variables["COST"])
        assert(len(unique_costs) == 1)
        assert(unique_costs[0] == COST)
    if DEPTH:
        if isinstance(DEPTH, dict):
            assert(np.all(individual_variables.apply(lambda row: row["DEPTH"] == DEPTH[row["codeversion"]][int(row["cond"])],axis=1)))
        else:
            unique_costs = np.unique(individual_variables["DEPTH"])
            assert(len(unique_costs) == 1)
            assert(unique_costs[0] == DEPTH)
    print(finished_df.groupby(["DEPTH", "COST"]).mean())
else:
    print(finished_df.groupby(["cond"]).mean())

           index  counterbalance  bonus    status          pid  DEPTH  COST  \
cond                                                                          
0      41.130435             0.0    0.0  3.782609  1661.565217    0.0   1.0   
1      84.454545             0.0    0.0  3.000000  1414.090909    0.0   1.0   
2      90.000000             0.0    0.0  3.000000  1419.636364    0.0   1.0   
3     108.900000             0.0    0.0  3.000000  1361.300000    0.0   1.0   
4     104.500000             0.0    0.0  3.000000  1410.000000    0.0   1.0   
5     114.090909             0.0    0.0  3.000000  1395.454545    0.0   1.0   
6     120.111111             0.0    0.0  3.000000  1396.111111    0.0   1.0   
7     129.300000             0.0    0.0  3.000000  1382.100000    0.0   1.0   
8     134.500000             0.0    0.0  3.000000  1387.300000    0.0   1.0   
9     125.181818             0.0    0.0  3.000000  1455.363636    0.0   1.0   

      MIN_TIME  inspectCost  bonusRate  branching  

In [7]:
time_fields = {"startTime":lambda date: np.nan if not isinstance(date,str) else datetime.strptime(date[:15], "%a %b %d %Y") ,"beginhit":lambda date: np.nan if  not isinstance(date,str) else datetime.fromisoformat(date.split(" ")[0]), "beginexp" : lambda date: np.nan if  not isinstance(date,str) else datetime.fromisoformat(date.split(" ")[0]), "endhit" : lambda date: np.nan if  not isinstance(date,str) else datetime.fromisoformat(date.split(" ")[0])}

for time_field, time_func in time_fields.items():
    individual_variables[time_field] = individual_variables[time_field].apply(time_func)

individual_variables["beginhit"].describe()

/tmp/ipykernel_135841/3025913967.py:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  individual_variables["beginhit"].describe()


count                     233
unique                      8
top       2020-10-23 00:00:00
freq                       91
first     2020-10-07 00:00:00
last      2020-11-03 00:00:00
Name: beginhit, dtype: object

In [8]:
if not isinstance(old_experiment, list):
    old_experiment = [old_experiment] * len(sessions)

In [9]:
survey_texts = preprocess_survey_text(full_data["survey-text"])

if any(old_experiment):
    old_runs = np.asarray(sessions)[old_experiment]
    old_demographics, demo_text = get_old_demographics(survey_texts[survey_texts["run"].isin(old_runs)], experiment_specific_gender=experiment_specific_mapping, manual_age_mapping=manual_age_mapping)
    print(demo_text)

new_runs =  np.asarray(sessions)[[not ex for ex in old_experiment]]
html_survey = full_data["survey-html-form"]
if len(html_survey_names) > 0:
    if "name" not in html_survey:
        html_survey["name"] = np.nan

    html_survey["name"] = html_survey.apply(
        lambda row: get_mouselab_quiz_name(row, html_survey_names[row["run"]]) if not isinstance(row["name"], str) else row["name"],
        axis=1)
    demographics, demo_text = process_html_demographics(html_survey[(html_survey["name"] == "demographics")&(html_survey["run"].isin(new_runs))])
    full_data["survey-multi-choice"] = pd.concat([full_data["survey-multi-choice"], html_survey[html_survey["name"] != "demographics"]])
else:
    demographics, demo_text = process_html_demographics(full_data["survey-html-form"])

print(demo_text)


if any(old_experiment):
    gender_name, gender_count = np.unique(pd.concat([old_demographics["Q2"],demographics["gender"]]), return_counts=True)
    ages = pd.concat([old_demographics["Q1"],demographics["age"]]).apply(int)
    print(get_demo_string(ages, gender_count, gender_name))

5 females, 8 males; median age 40, age range 24-63
37 females, 54 males; median age 31, age range 18-69
42 females, 62 males; median age 32, age range 18-69


In [10]:
mouselab_datas = preprocess_mouselab_data(full_data["mouselab-mdp"],trials_per_block,ground_truths)

mouselab_datas = mouselab_datas.merge(individual_variables, how="left", on=["pid", "run"])

# path may contain a bunch of 0s at the start due to miscoding
mouselab_datas["path"] = mouselab_datas["path"].apply(lambda path : eval(path)[-3:])

node_classification = {key : [str(node) for node in val] for key, val in node_classification.items()}
mouselab_datas = add_click_count_columns(mouselab_datas, node_classification)

In [11]:
#TODO would be nice to refactor this out and import it
def expand_range_dictionary(input_dictionary):
    trial_to_block = {}
    for block, trial_range in input_dictionary.items():
        if isinstance(trial_range, str):
            for trial_index in eval(trial_range):
                trial_to_block[trial_index] = block
        else:
            trial_to_block[block] = expand_range_dictionary(trial_range)
    return trial_to_block

if ranges_to_extract:
    trial_to_block = expand_range_dictionary(ranges_to_extract)

    mouselab_datas["block"] = mouselab_datas.apply(lambda row: trial_to_block[row["run"]][row["trial_index"]] if row["run"] in trial_to_block else trial_to_block[row["trial_index"]], axis=1)

In [12]:
questionnaires = full_data["survey-multi-choice"]

if "name" not in questionnaires:
    questionnaires["name"] = np.nan

questionnaires["name"] = questionnaires.apply(
    lambda row: get_mouselab_quiz_name(row, mouselab_mapping) if not isinstance(row["name"], str) else row["name"],
    axis=1)

mouselab_questionnaires = questionnaires[questionnaires["name"].isin(mouselab_mapping.values())].reset_index()
questionnaires = questionnaires[~questionnaires["name"].isin(mouselab_mapping.values())].reset_index()

mouselab_questionnaires["name"] = mouselab_questionnaires.apply(lambda row: get_mouselab_quiz_name(row, mouselab_mapping) if not isinstance(row["name"], str) else row["name"],axis=1)
mouselab_questionnaires = score_mouselab_questionnaires(mouselab_questionnaires, mouselab_quiz_solutions, mouselab_column_identifier)
mouselab_quiz = mouselab_questionnaires.drop_duplicates(["pid","question_id"], keep="last")

pivoted_mouselab_quiz = mouselab_quiz.pivot_table(values="score", index=["pid","run"], columns="question_id")

In [13]:
if len(questionnaires)>0:
    # remove potentially sensitive questionnaire
    questionnaires = questionnaires[~questionnaires["name"].isin(['decision-outcome'])]
    questionnaire_files = {}
    for session in sessions:
        with open(inputs_path.joinpath(f"questionnaire_files/solutions.pkl"), "rb") as f:
            questionnaires_presentation = pickle.load(f)
        questionnaire_files[session] = {**questionnaires_presentation, **mouselab_quiz_solutions, **{"crt":crt_quiz_solutions}}
    for run in sessions:
        questionnaire_files[run]['UPPS-P']["catch.2"] = {'Agree Strongly': 0,
              'Agree Some': 0,
              'Disagree Some': 1,
              'Disagree Strongly': 0}
    questionnaires["open_ended"] = questionnaires["name"].apply(lambda entry: entry in ["crt", "mcq"])#TODO
    questionnaires["responses"] = questionnaires["responses"].apply(lambda entry: entry.replace("null", "None").replace("”","").replace("“","") if isinstance(entry, str) else entry)

# Add questionnaire names that we're missing in runs QuestPilot1 and QuestPilot 2 (by elimination)

In [14]:
questionnaires["responses"]=questionnaires["responses"].apply(eval)

In [15]:
# add questionnaire names that were missing (QuestPilot1, QuestPilot2)
possible_questionnaires = questionnaires[~pd.isnull(questionnaires["name"])]["name"].unique()

all_possible_answers = {}
for quest in questionnaire_files["QuestPilot3"]:
    all_possible_answers[quest] = set()
    for item in questionnaire_files["QuestPilot3"][quest]:
        # only non-open ended questionnaires
        if isinstance(questionnaire_files["QuestPilot3"][quest][item], dict):
            all_possible_answers[quest] = all_possible_answers[quest].union(set(questionnaire_files["QuestPilot3"][quest][item].keys()))
    if len(all_possible_answers[quest])==0:
        del all_possible_answers[quest]

all_possible_answers["MCQ"] = ['larger reward in the specified number of day s',
       'larger reward in the specified number of days',
       'smaller reward today']

second_iq_question = ['Richard is as tall as Matt', "It's impossible to tell",
       'Richard is taller than Matt', 'Richard is shorter than Matt']

In [16]:
def label_iq(row):
    if len(row["responses"]) > 1:
        if row["responses"][1] in second_iq_question:
            return "IQ"
        else:
            return row["name"]
    else:
        return row["name"]
    
def remove_null_quest_ids(row):
    if isinstance(row["question_id"], str):
        if "-1" in row["question_id"] or row["question_id"] == "nan":
            return np.nan
        else:
            return row["question_id"]
    else:
        return row["question_id"]
    
def add_questionnaire_id(row, all_possible_answers, possible_questionnaires, questionnaires_for_pid):
    if isinstance(row["name"], str):
        return row["name"]
    else:
        unique_responses = set(row["responses"])
        
        # only non-open ended questionnaires
        multiple_choice = [quest for quest in possible_questionnaires if quest in all_possible_answers.keys()]
        
        possible_questionnaires_for_pid = set(multiple_choice).difference(questionnaires_for_pid[row["pid"]])

        fitting_questionnaires_for_pid = [quest for quest in possible_questionnaires_for_pid if unique_responses.issubset(all_possible_answers[quest])]
        
        if len(fitting_questionnaires_for_pid)==1:
            return fitting_questionnaires_for_pid[0]
        else:
            return row["name"]
        
def fix_question_id(row):
    if not pd.isnull(row["question_id"]):
        return row["question_id"]
    else:
        if pd.isnull(row["name"]):
            return row["question_id"]
        else:
            if (row["name"] == "MCQ") & (row["run"] == "QuestPilot1"):
                return str(eval(questionnaire_id_mapping[row["name"]])[:-1])
            else:
                return questionnaire_id_mapping[row["name"]]

In [17]:
questionnaires["question_id"] = questionnaires.apply(remove_null_quest_ids, axis=1)
questionnaires["responses"] = questionnaires["responses"].apply(lambda responses : list(responses.values()) if isinstance(responses, dict) else responses)

questionnaires["name"] =questionnaires.apply(label_iq, axis=1)

# needs to be more than 1, 2 rows are unclassifiable
for _ in range(3):
    questionnaires_for_pid = {}
    for pid in questionnaires["pid"].unique():
        questionnaires_for_pid[pid] = questionnaires[questionnaires["pid"]==pid]["name"].unique()
    questionnaires["name"] = questionnaires.apply(lambda row: add_questionnaire_id(row, all_possible_answers, possible_questionnaires, questionnaires_for_pid), axis=1)

# Fix question ids for scoring

In [18]:
questionnaire_id_mapping = questionnaires[questionnaires["run"]=="QuestPilot3"][["name","question_id"]].set_index("name").to_dict()['question_id']
questionnaires["question_id"] = questionnaires.apply(fix_question_id  , axis=1)

questionnaires["responses"]=questionnaires["responses"].apply(str)

In [19]:
scored_questionnaire_df = score_generic_questionnaires(questionnaires, questionnaire_files, group_identifier="name", default_open_ended={"crt": "other"})

In [20]:
# one catch trial sometimes misnamed
scored_questionnaire_df["question_id"] = scored_questionnaire_df.apply(
    lambda row: "catch.1"
    if (row["name"] == "UPPS-P") & (row["question_id"] == "catch.2")
    else row["question_id"],
    axis=1,
)

In [21]:
# convert scores to numeric when numeric
scored_questionnaire_df["score"] = scored_questionnaire_df["score"].apply(lambda score: int(score) if str(score).isnumeric() else score)

In [22]:
quiz_passers = get_quiz_passer_ids(mouselab_questionnaires, max_attempts=max_attempts, passing_score=passing_score, identifying_columns = ["pid", "run"])
passed_all_quizzes = list(set.intersection(*map(set,quiz_passers.values())))
print(len(passed_all_quizzes))
pivoted_mouselab_quiz["passed_quizzes"] = 0
pivoted_mouselab_quiz.loc[passed_all_quizzes, "passed_quizzes"] = 1

/home/vfelso/github/irl-project/venv/lib/python3.8/site-packages/download_tools/plugins/survey_multi_choice.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data["attempt_num"] = survey_data["internal_node_id"].apply(


102


In [23]:
full_demographics = pd.concat([demographics, old_demographics[["Q1", "Q2"]].rename(columns = {"Q1": "age", "Q2": "gender"})
                                   ],join="outer", sort=False)

# only include participants who really finished experiment, and were paid
passed_all_quizzes = [part for part in passed_all_quizzes if part[0] in full_demographics.index]

In [24]:
quiz_and_demo = pivoted_mouselab_quiz.join(full_demographics)
quiz_and_demo = quiz_and_demo.merge(individual_variables, how="left", on=["pid", "run"])

quiz_and_demo_subselection = quiz_and_demo[quiz_and_demo.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)]

quiz_and_demo_subselection["gender"]=quiz_and_demo_subselection["gender"].replace(np.nan, "participants with no demographic")
ages = [int(age) for age in quiz_and_demo_subselection["age"] if not pd.isnull(age)]

gender_values, gender_counts = np.unique(quiz_and_demo_subselection["gender"].values, return_counts = True)
print(get_demo_string(ages, gender_counts, gender_values))

if len(questionnaires)>0:
        scored_questionnaire_df[scored_questionnaire_df.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("questionnaires.csv"))
quiz_and_demo[quiz_and_demo.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("quiz-and-demo.csv"))
mouselab_datas[mouselab_datas.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("mouselab-mdp.csv"))
survey_texts[survey_texts.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("survey-text.csv"))
individual_variables[individual_variables.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("individual-variables.csv"))

38 females, 57 males; median age 32, age range 20-69


/tmp/ipykernel_135841/3685819383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_and_demo_subselection["gender"]=quiz_and_demo_subselection["gender"].replace(np.nan, "participants with no demographic")


In [25]:
if num_parts:
    #TODO only supports one digit num_parts
    for part_num in range(1, num_parts +1):
        curr_runs = [run for run in finished_df["run"].unique() if run.endswith(f"Part{part_num}")]
        curr_runs_df = finished_df[finished_df["run"].isin(curr_runs)]
        median_info = curr_runs_df[curr_runs_df.apply(lambda row: (row["pid"], row["run"][:-1]+str(1)) in \
                                     passed_all_quizzes, axis=1)].median()
        print(f"median time: {median_info['time_diff']:.2f}, median bonus: {median_info['displayed_bonus']:.2f}")
else:
    median_info = finished_df[finished_df.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].median()
    print(f"median time: {median_info['time_diff']:.2f}, median bonus: {median_info['displayed_bonus']:.2f}")

median time: 39.88, median bonus: 3.86


/tmp/ipykernel_135841/193806058.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  median_info = finished_df[finished_df.apply(lambda row: (row["pid"], row["run"]) in \


In [26]:
questionnaires[questionnaires.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].groupby(["name"]).count()["pid"]

name
AES         93
AUDIT       95
BIS         95
CFC         95
EAT         95
IQ          87
IUS_12       8
LSAS_A      95
LSAS_B      95
MCQ         95
OCIR        79
SCZ         87
STAI        95
STICSA_S     8
STICSA_T     8
UPPS-P      95
Zhung       95
Name: pid, dtype: int64

In [27]:
valid_questionnaires = scored_questionnaire_df[scored_questionnaire_df.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)]

individual_items = valid_questionnaires.pivot_table(
        index=["pid"], columns="question_id", values="score"
    )
individual_items = individual_items.join(full_demographics[full_demographics.columns.difference(["female", "male"])], on="pid")
individual_items.to_csv(processed_data_path.joinpath("individual_items.csv"))

summed_scores = valid_questionnaires.groupby(["pid", "name"]).sum()["score"].reset_index()
combined_scores = summed_scores.pivot_table(
    index=["pid"], columns="name", values="score"
)
combined_scores = combined_scores.join(full_demographics[full_demographics.columns.difference(["female", "male"])], on="pid")
combined_scores.to_csv(processed_data_path.joinpath("combined_scores.csv"))